In [51]:
import indexer as ind
import requests
reload(ind)
url = ind.es_url + "/" + ind.tmdb

In [52]:
docs = [{"title":"the bunny thing"},{"title":"the bunny thing"}]
ind.index_docs(docs)

200
Indexing 2 movies


In [55]:
requests.get(url + "/_search").text

u'{"took":2,"timed_out":false,"_shards":{"total":1,"successful":1,"failed":0},"hits":{"total":2,"max_score":1.0,"hits":[{"_index":"tmdb","_type":"movie","_id":"0","_score":1.0,"_source":{"title": "the bunny thing"}},{"_index":"tmdb","_type":"movie","_id":"1","_score":1.0,"_source":{"title": "the bunny thing"}}]}}'

In [86]:
resp = requests.delete(url)
resp = requests.put(url, data="""
{
  "settings" : {
    "index" : {
      "number_of_shards" : 1,
      "number_of_replicas" : 0
    },
    "analysis": {
      "analyzer": {
        "fulltext_analyzer": {
          "type": "custom",
          "tokenizer": "whitespace",
          "filter": [
            "lowercase",
            "type_as_payload"
          ]
        },
        "fulltext_analyzer2": {
          "type": "custom",
          "tokenizer": "whitespace",
          "filter": [
            "lowercase",
            "type_as_payload"
          ]
        }
      }
    }
  }
}
""")
if not resp.ok:
    print resp.text
else:
    resp = requests.get(url + "/_analyze?analyzer=standard", 
                        data="Fire with Fire")
    if resp.ok:
        tokens = resp.json()['tokens']
        for token in tokens:
            print token
    

{u'end_offset': 4, u'token': u'fire', u'type': u'<ALPHANUM>', u'start_offset': 0, u'position': 1}
{u'end_offset': 9, u'token': u'with', u'type': u'<ALPHANUM>', u'start_offset': 5, u'position': 2}
{u'end_offset': 14, u'token': u'fire', u'type': u'<ALPHANUM>', u'start_offset': 10, u'position': 3}
